<a href="https://colab.research.google.com/github/youssef1234-pixel/Python_tutorials/blob/main/OOP_Project(Online_Shopping_System).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class User:
    def __init__(self, user_id, username):
        self.user_id = user_id
        self.username = username

    def show_dashboard(self):
        pass


class Admin(User):
    def __init__(self, user_id, username):
        super().__init__(user_id, username)

    def add_product(self, inventory, product):
        inventory.add_product(product)
        print(f"Admin {self.username} added product: {product.name}")

    def remove_product(self, inventory, product_id):
        inventory.remove_product(product_id)
        print(f"Admin {self.username} removed product with ID: {product_id}")

    def show_dashboard(self):
        print(f"Admin Dashboard for {self.username}")



class Customer(User):
    def __init__(self, user_id, username):
        super().__init__(user_id, username)
        self.cart = Cart()
        self.order_history = []

    def browse_products(self, inventory):
        print(f"{self.username}'s Product Browsing:")
        inventory.display_products()

    def add_to_cart(self, product, quantity):
        self.cart.add_item(product, quantity)
        print(f"{quantity} of {product.name} added to {self.username}'s cart.")

    def view_cart(self):
        self.cart.display_cart()

    def place_order(self):
        if not self.cart.items:
            print("Your cart is empty. Cannot place an order.")
            return None

        order = Order(self.user_id, self.cart.items)
        self.order_history.append(order)
        print(f"Order placed successfully for {self.username}. Total: ${order.get_total():.2f}")

        self._store_order_to_file(order)

        self.cart = Cart()
        return order

    def view_order_history(self):
        print(f"{self.username}'s Order History:")
        if not self.order_history:
            print("No previous orders.")
        else:
            for order in self.order_history:
                order.display_order()

    def show_dashboard(self):
        print(f"Customer Dashboard for {self.username}")
        self.view_cart()
        self.view_order_history()

    def _store_order_to_file(self, order):
        try:
            with open('orders.txt', 'a') as f:
                f.write(f"Order ID: {order.order_id}, Customer ID: {order.customer_id}, Total: {order.get_total():.2f}, Items: {order.items}\n")
        except Exception as e:
            print(f"Error storing order to file: {e}")


class Product:
    def __init__(self, product_id, name, price, stock):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.stock = stock

    def __repr__(self):
        return f"Product(ID: {self.product_id}, Name: {self.name}, Price: ${self.price:.2f}, Stock: {self.stock})"


class Cart:
    def __init__(self):
        self.items = {}

    def add_item(self, product, quantity):
        if product.stock >= quantity:
            if product.product_id in self.items:
                self.items[product.product_id] += quantity
            else:
                self.items[product.product_id] = quantity
            product.stock -= quantity
        else:
            print(f"Not enough stock for {product.name}. Available stock: {product.stock}")

    def remove_item(self, product_id, quantity):
        if product_id in self.items:
            if self.items[product_id] >= quantity:
                self.items[product_id] -= quantity
                if self.items[product_id] == 0:
                    del self.items[product_id]

            else:
                print(f"You only have {self.items[product_id]} of this item in your cart.")
        else:
            print("Item not found in cart.")

    def get_total(self, inventory):
        total = 0
        for product_id, quantity in self.items.items():
            product = inventory.get_product(product_id)
            if product:
                total += product.price * quantity
        return total

    def display_cart(self):
        print("Shopping Cart:")
        if not self.items:
            print("Cart is empty.")
        else:
            for product_id, quantity in self.items.items():
                print(f"  Product ID: {product_id}, Quantity: {quantity}")


class Order:
    order_counter = 0

    def __init__(self, customer_id, items):
        Order.order_counter += 1
        self.order_id = Order.order_counter
        self.customer_id = customer_id
        self.items = items.copy()

    def get_total(self, inventory=None):
        total = 0
        if inventory:
            for product_id, quantity in self.items.items():
                product = inventory.get_product(product_id)
                if product:
                    total += product.price * quantity
        else:
             print("Warning: Cannot calculate accurate order total without inventory context in Order class.")

             total = sum(self.items.values())

        return total

    def display_order(self):
        print(f"  Order ID: {self.order_id}, Customer ID: {self.customer_id}, Items: {self.items}")



class Inventory:
    def __init__(self):
        self.products = {}

    def add_product(self, product):
        self.products[product.product_id] = product

    def remove_product(self, product_id):
        if product_id in self.products:
            del self.products[product_id]
        else:
            print(f"Product with ID {product_id} not found in inventory.")

    def get_product(self, product_id):
        return self.products.get(product_id)

    def display_products(self):
        print("Inventory Products:")
        if not self.products:
            print("No products in inventory.")
        else:
            for product_id, product in self.products.items():
                print(f"- {product}")


inventory = Inventory()


admin1 = Admin(1, "AliceAdmin")


product1 = Product(101, "Laptop", 1200.00, 10)
product2 = Product(102, "Mouse", 25.00, 50)
product3 = Product(103, "Keyboard", 75.00, 30)

admin1.add_product(inventory, product1)
admin1.add_product(inventory, product2)
admin1.add_product(inventory, product3)


inventory.display_products()


customer1 = Customer(201, "BobCustomer")


customer1.browse_products(inventory)


customer1.add_to_cart(product1, 1)
customer1.add_to_cart(product2, 2)
customer1.add_to_cart(product3, 1)
customer1.add_to_cart(product1, 1)

customer1.view_cart()


order1 = customer1.place_order()
if order1:

    print(f"Calculated Order 1 Total (using inventory): ${order1.get_total(inventory):.2f}")



customer1.view_order_history()


inventory.display_products()


customer2 = Customer(202, "CharlieCustomer")
customer2.browse_products(inventory)
customer2.add_to_cart(product2, 5)
customer2.add_to_cart(product3, 2)
customer2.view_cart()
order2 = customer2.place_order()
if order2:
     print(f"Calculated Order 2 Total (using inventory): ${order2.get_total(inventory):.2f}")


customer2.view_order_history()


admin1.remove_product(102)
inventory.display_products()


print("\n--- Dashboards ---")
admin1.show_dashboard()
customer1.show_dashboard()
customer2.show_dashboard()


print("\n--- Orders from File ---")
try:
    with open('orders.txt', 'r') as f:
        print(f.read())
except FileNotFoundError:
    print("orders.txt not found.")


Admin AliceAdmin added product: Laptop
Admin AliceAdmin added product: Mouse
Admin AliceAdmin added product: Keyboard
Inventory Products:
- Product(ID: 101, Name: Laptop, Price: $1200.00, Stock: 10)
- Product(ID: 102, Name: Mouse, Price: $25.00, Stock: 50)
- Product(ID: 103, Name: Keyboard, Price: $75.00, Stock: 30)
BobCustomer's Product Browsing:
Inventory Products:
- Product(ID: 101, Name: Laptop, Price: $1200.00, Stock: 10)
- Product(ID: 102, Name: Mouse, Price: $25.00, Stock: 50)
- Product(ID: 103, Name: Keyboard, Price: $75.00, Stock: 30)
1 of Laptop added to BobCustomer's cart.
2 of Mouse added to BobCustomer's cart.
1 of Keyboard added to BobCustomer's cart.
1 of Laptop added to BobCustomer's cart.
Shopping Cart:
  Product ID: 101, Quantity: 2
  Product ID: 102, Quantity: 2
  Product ID: 103, Quantity: 1
Order placed successfully for BobCustomer. Total: $5.00
Calculated Order 1 Total (using inventory): $2525.00
BobCustomer's Order History:
  Order ID: 1, Customer ID: 201, Items:

TypeError: Admin.remove_product() missing 1 required positional argument: 'product_id'